## AZURE SQL - DATABASE POPULATION
SCHOOL DETAILS DATA
Make sure that you have the latest ODBC driver
https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16#download-for-windows

In [1]:
! pip install pyodbc

In [ ]:
!  pip install sqlalchemy

In [1]:
import pandas as pd
import json
import pyodbc
import sqlalchemy as db
import urllib

exec(open('azure_credentials.py').read())

In [2]:
username

'aid2023adminlog'

In [3]:
# Load csv
details_schoolNumbers = pd.read_csv('details_schoolNumbers.csv')

In [5]:
details_schoolNumbers.head(3)

,idSchool,studyFormationsCount,studentsCount,personnelCount
0,11269910,4,131,7
1,11256876,0,0,0
2,11256865,0,0,0


#### CONNECTION DETAILS

In [4]:
database_name = "AID2023"
driver = '{ODBC Driver 18 for SQL Server}'
server_name = "aid-2023-srv.database.windows.net"

#### CONNECT TO AZURE DATABASE
Driver={ODBC Driver 18 for SQL Server};Server=tcp:aid-2023-srv.database.windows.net,1433;Database=AID2023;Uid=aid2023adminlog;Pwd={your_password_here};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;

In [5]:
# Create the server url
server = 'tcp:{server_name},1433'.format(server_name = server_name)

#create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
    driver = driver,
    server=server,
    database=database_name,
    username=username,
    password=password
)                              

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
cursor: pyodbc.Cursor = cnxn.cursor() 

#execute a query with the cursor to check if the connection is maade
print("Connection is open") if cursor.execute("SELECT 1") else print(f"Connection is closed: {e}")



Connection is open


### USE SQL ALCHEMY TO LOAD THE DATAFRAMES

In [11]:
#obtain the parameters from the pyodbc connection string
params = urllib.parse.quote_plus(connection_string)
#create the engine
sqlalchemy_engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)


In [16]:
sqlalchemy_engine

Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BSERVER%3Dtcp%3Aaid-2023-srv.database.windows.net%2C1433%3BDATABASE%3DAID2023%3BUID%3Daid2023adminlog%3BPWD%3D%40Aidr2023sql%3Bcharset%3Dutf8)

### DETAILS DATA

#### SCHOOL NUMBERS DF TO SQL

In [17]:
details_schoolNumbers.columns

Index(['idSchool', 'studyFormationsCount', 'studentsCount', 'personnelCount'], dtype='object')

In [6]:

# Get the list of installed drivers
drivers = [driver for driver in pyodbc.drivers()]

# Print the list of drivers
print("Installed ODBC Drivers:")
for driver in drivers:
    print(driver)

Installed ODBC Drivers:
SQL Server
PostgreSQL ANSI(x64)
PostgreSQL Unicode(x64)
ODBC Driver 17 for SQL Server
MySQL ODBC 8.0 ANSI Driver
MySQL ODBC 8.0 Unicode Driver
ODBC Driver 18 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)


In [8]:
details_schoolNumbers.head()

,idSchool,studyFormationsCount,studentsCount,personnelCount
0,11269910,4,131,7
1,11256876,0,0,0
2,11256865,0,0,0
3,11284187,0,0,0
4,11270113,0,0,0


#### UPLOAD TO SQL

In [ ]:
#insert the values form details df into sql table
details_schoolNumbers.to_sql("Details_Vlad", sqlalchemy_engine, if_exists='replace', index=False) # "Details" = the name of the table you want to create in the database


#### QUERY DB

In [ ]:
sql_query = "SELECT TOP 10 * FROM Details_Vlad" 
# execute
cursor.execute(sql_query)
top10 = cursor.fetchall()